# PySpark 1

Apache Spark는 large-scale의 데이터 프로세싱을 위한 통합된 분석 엔진이다.  
  
분산된 여러 대의 노드에서 연산을 할 수 있게 해주는 범용 분산 클러스터링 플랫폼.  
Memery Hadoop이라고도 불리는데, 기존의 하둡이 Map & Reduce 작업을 디스크 기반으로 하기 때문에 느려지는 성능을 메모리 기반으로 옮겨서 고속화 하고자 하는데서 출발함. 

['조대협의 블로그 : Apache Spark'에 대한 설명](http://bcho.tistory.com/1023?category=563141)

Spark program은 일반적으로 "Driver Program"이라고 하는데, 이러한 driver program은 여러개의 병렬적인 작업으로 나눠져서 Spark의 Worker Node(서버)에 있는 Executor(프로세스)에서 실행된다.  

1. SparkContext가 SparkClusterManager에 접속한다. 이 클러스터 메니져는 스팍 자체의 클러스터 메니져가 될 수 도 있고 Mesos,YARN 등이 될 수 있다. 이 클러스터 메니저를 통해서 가용한 Excutor 들을 할당 받는다
2. Excutor를 할당 받으면, 각각의 Executor들에게 수행할 코드를 보낸다.
3. 다음으로 각 Excutor 안에서 Task에서 로직을 수행한다.


출처: http://bcho.tistory.com/1025?category=563141 [조대협의 블로그]


In [23]:
import findspark
# pyspark를 import할 수 있도록 찾아주는 라이브러리이다.

In [24]:
findspark.init() # pyspark를 일반적인 라이브러리처럼 import할 수 있게 해줌.

In [25]:
from pyspark import SparkContext
# Spark 기능에 대한 기본 entry-point(진입점).
# SparkContext는 Spark Cluster에 대한 연결을 나타낸다.

In [26]:
sc = SparkContext.getOrCreate() # default spark context 생성

In [27]:
sc.stop() 
# sparkContext를 여러개 만들면 안돌아간다. 그래서 새로운 context생성시 먼저 stop을 해줌.

In [28]:
sc = SparkContext(master = 'local[2]', appName = 'Spark Test')
# master: 연결할 Cluster URL(e.g. mesos://host:port, spark://host:port, local[4]).
# appName: cluster web UI에 띄울 작업의 이름

In [29]:
sc

<SparkContext master=local[2] appName=Spark Test>

In [30]:
#rdd = sc.parallelize([1,2,3])

In [31]:
#rdd.collect()

__RDD__ (__R__esilient __D__istributed __D__ataset)  
RDD는 여러 분산 노드에 걸쳐서 저장되는 변경 불가능한 데이터(객체)의 집합이며,  
각각의 RDD는 여러개의 파티션으로 분리되어 서로 다른 노드에서 실행된다.  
쉽게 말해서 스파크 내에 저장된 데이터를 의미한다.  

RDD는 변경이 불가하기 때문에 변경하기 위해선 새로 다시 생성해야 한다.    
외부로부터 데이터를 로딩하거나 코드에서 생성된 데이터를 저장함으로써 생성 가능.  
  
RDD에서는 2가지의 operation만 지원해준다.
 - Transformation(변환) : 기존의 RDD 데이타를 변경하여 새로운 RDD 데이타를 생성해내는 것. 흔한 케이스는 filter와 같이 특정 데이타만 뽑아 내거나 map 함수 처럼, 데이타를 분산 배치 하는 것 등을 들 수 있다.
 - Action(액션) : RDD 값을 기반으로 무엇인가를 계산해서(computation) 결과를 생성해 내는것으로 가장 쉬운 예로는 count()와 같은 operation들을 들 수 있다.


In [32]:
sc.stop() # SparkContext를 중지시킨다.

In [33]:
from pyspark import SparkConf
# SparkConf : Configuration for Spark App. Spark 어플을 설정하는데 쓰인다.
# 다양한 Spark 파라미터들을 key-value 쌍으로 세팅하는데 쓰인다.

In [34]:
conf = SparkConf()
# spark.*로 부터 value들, Java system property들도 또한 로딩해준다.

In [35]:
conf.setMaster("local[2]").setAppName("Conf")
# 연결할 marster URL을 "local[2]", AppName을 "Conf"로 지정해준다.

In [36]:
sc = SparkContext(conf=conf)
# conf : A L{SparkConf} object setting Spark properties.
# conf에 세팅된 Spark property를 SparkContext에 설정해서 객체 생성.

In [37]:
rdd = sc.parallelize([1,2,3,4,5]) # rdd 생성
# 로컬 파이썬 컬렉션을 RDD를 생성하기 위해 분산시킨다.
# 만약 input이 성능에 대한 range를 표현한다면 xrange를 사용하는 것을 추천함

In [38]:
rdd

ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:184

In [39]:
rdd.collect() # 기본적인 rdd를 생성한 것이다.
# collect() : RDD내에 있는 모든 element들을 포함하는 리스트를 리턴해준다.

[1, 2, 3, 4, 5]

In [40]:
rdd.glom().collect() # partition단위로 작업을 한 것이다.
# glom() : 모든 element들을 각각의 파티션에 맞게 리스트로 합쳐서 만들어진 RDD 반환

[[1, 2], [3, 4, 5]]

In [41]:
len(rdd.glom().collect()) 
# partition을 2개로 만들어버림. 나중에 파일 2개로 반환해줄것이다.

2

In [42]:
rdd = sc.parallelize([1,2,3,4,5], 3) # partition을 3개로 나눠서 rdd 객체 만들어줌
# 기본적으로 partition 2개로 만들어줌.

In [49]:
text = sc.parallelize(open("./sample/text-test.txt", encoding='utf-8').read())

In [50]:
text.glom().collect() # partition을 나눠서 한 글자씩 받아옴.
#glom은 각 partition마다 element들을 나눠서 리턴해준다. = transformation

[['h', 'e', 'l', 'l', 'o'], ['\n', 't', 'h', 'i', 's']]

In [51]:
text.collect() # 한 글자씩 다 받아옴.
#collect는 모든 element들을 다 받아오는 것이다. = action

['h', 'e', 'l', 'l', 'o', '\n', 't', 'h', 'i', 's']

In [52]:
rdd = sc.parallelize(["b", "a", "c"], 3) 
# partition 3개로 나눠서 해당 리스트 rdd 객체 생성

In [53]:
rddMap = rdd.map(lambda x:(x,1)) # mapping해줌.
# 특정 형태로 출력해줌.

In [54]:
sorted(rddMap.glom().collect()) 
# partition 3개로 나뉘어져 있는 mapping결과를 정렬해서 출력해줌.

[[('a', 1)], [('b', 1)], [('c', 1)]]

In [55]:
rdd = sc.parallelize([1,2,3,4,5])
rdd.filter(lambda x:x%2==0).collect() # rdd객체에서 filtering한 객체를 출력해줌.
# 나누기 해서 나머지가 0이 나오는 것들만 보여줌.

[2, 4]

In [61]:
rdd = sc.parallelize([2,3,4])
rdd = rdd.map(lambda x:range(1, x)) # lambda식으로 mapping

In [63]:
rdd.glom().collect() # 기본적으로 partition2개로 나눠진다.

[[range(1, 2)], [range(1, 3), range(1, 4)]]

In [64]:
# flatMap의 경우 각 RDD에 적용시 partition단위로 출력하며, 없는 데이터는 삭제해준다.

rdd = sc.parallelize([2,3,4])
rdd.flatMap(lambda x:range(1,x)).collect()

# flatMap의 경우 Mapping해준뒤 1열로 하나의 리스트에 한개씩 쭉 나열해준다.

[1, 1, 2, 1, 2, 3]

In [65]:
rdd = sc.parallelize(["Roses are red", "Violets are blue"])

In [66]:
rdd.map(lambda x:x.split()).collect()

[['Roses', 'are', 'red'], ['Violets', 'are', 'blue']]

In [67]:
rdd = sc.parallelize(["Roses are red", "Violets are blue"])
rdd.flatMap(lambda x:x.split()).collect()
# flatMap의 경우 Mapping해준뒤 1열로 하나의 리스트에 한개씩 쭉 나열해준다.

['Roses', 'are', 'red', 'Violets', 'are', 'blue']

__map vs flatMap__  

__map__ : RDD의 각 요소에 주어진 함수를 적용하여 새로운 RDD를 반환합니다. map 함수는 하나의 항목 만 반환합니다.  
  
__flatMap__ : map 와 마찬가지로 RDD의 각 요소에 함수를 적용하여 새로운 RDD를 반환하지만 출력은 병합됩니다.  


map과 flatMap은 비슷하다. 즉 입력 RDD로부터 한 줄을 가져 와서 그 위에 함수를 적용한다. 이들이 다른 점은 map의 함수는 하나의 요소 만 반환하는 반면 flatMap의 함수는 요소 목록 (0 이상)을 반복자로 반환 할 수 있다는 점입니다.  
  
또한 flatMap의 출력이 병합됩니다. flatMap의 함수가 요소 목록을 반환하더라도 flatMap은 목록의 모든 요소가 플랫이 아닌 (목록이 아닌) RDD를 반환합니다.  

[map과 flatMap의 차이점 참고 링크](https://code.i-harness.com/ko-kr/q/1550b82)

In [68]:
sc.parallelize([1,1,2,3]).distinct().collect()
#distinct()는 중복된 결과를 없애준다.

[2, 1, 3]

In [69]:
rdd = sc.parallelize(range(100), 4) # partition을 4개로 줌.

In [70]:
#sample은 일부 샘플만 추출해서 가져온다. RDD의 subset을 추출해줌.
subset = rdd.sample(False, 0.1, 12)
# 중복 False, 10% 비율, seed =12로 주고 랜덤으로 샘플링 rdd 객체 생성
# 다만 비율이 정확한 값이 아니라 대략 10%를 갖고 오는 것이다.

In [71]:
subset

PythonRDD[35] at RDD at PythonRDD.scala:49

In [72]:
subset.glom().collect() # 100개 중에 12개를 갖고 왔음을 알 수 있다.
# 비율이 대략적인 값을 나타냄을 알 수 있다.

[[8, 10, 15, 16, 19], [26, 32], [62, 63, 65, 72], [98]]

In [73]:
subset.count() # rdd 객체 내부의 element 개수 : action
# count()의 경우 spark내부 함수. 그래서 len()보다는 더 최적화되어 빠르다.

12

__JOIN__

In [74]:
x = sc.parallelize([("a",1), ("b", 4)])
y = sc.parallelize([("a",2)])

In [75]:
# Left Outer Join
sorted(x.leftOuterJoin(y).collect())

[('a', (1, 2)), ('b', (4, None))]

In [76]:
sorted(y.leftOuterJoin(x).collect())

[('a', (2, 1))]

In [77]:
# Right Outer Join
sorted(x.rightOuterJoin(y).collect())

[('a', (1, 2))]

In [78]:
# Join : 키 값이 같은 것들에 대해 join
sorted(x.join(y).collect())

[('a', (1, 2))]

In [79]:
# Intersection : 교집합해줌.
sorted(x.intersection(y).collect()) # 완전히 교집합 되는 것이 없어서 출력되는것이 없음.

[]

In [80]:
x = sc.parallelize([("a",1), ("b", 4)])
y = sc.parallelize([("a",2), ("b", 4)])
sorted(x.intersection(y).collect()) # ("b", 4)가 완전히 교집합 되기 때문에 출력됨.

[('b', 4)]

__repartition__  
: 파티션 개수가 다를 경우 join이나 zip할 때 오류가 발생할 수 있어  
그것을 방지하기 위해 다시 partition 개수를 수정해준다.  

In [81]:
rdd.glom().count() # 현재 4개의 partition을 가지고 있다.

4

In [82]:
rdd.repartition(10).glom().collect() #파티션 개수를 10개로 수정.

[[],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84],
 [20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94],
 [35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  70,
  71,
  72,
  73,
  74,
  95,
  96,
  97,
  98,
  99],
 [45, 46, 47, 48, 49],
 [],
 [],
 [],
 []]

http://localhost:4040/ 으로 들어갈 경우 spark에서 작업을 어떻게 수행했는지 볼수 있다.

__take__ : 하나의 partition에서 상위 몇가지의 element 리턴 = transformation  
__cache__: 따로 작업공간을 만들어줌. 차후에 꼭!! cache공간을 없애주어야함.  
그렇지 않는다면 프로세스 종료후에도 남아있는 경우가 있다.  
__takeSample__ : 특정샘플 몇개만 갖고 오는 것. = action  

In [83]:
rdd.take(3)

[0, 1, 2]

In [87]:
rdd_cache = rdd.cache()

In [88]:
rdd_cache

PythonRDD[34] at RDD at PythonRDD.scala:49

In [94]:
rdd.takeSample(False, 10, 123) # action 함수이기 때문에 따로 print해줄 필요가 없음.
# 중복 False, 10개, seed = 123

[26, 14, 70, 99, 60, 78, 80, 13, 33, 43]

__reduce__ : key, value 쌍으로 된 것들을 partition 단위로 reduce 시행.  

In [104]:
rdd = sc.parallelize([1,2,3,4,5],2)

In [105]:
rdd.glom().reduce(lambda x,y:x+y)
# [[1, 2] , [3, 4, 5]]
# [1, 2, 3, 4, 5]

[1, 2, 3, 4, 5]

In [108]:
#partition 개수에 따라서 생각하는 operation이 아닌 다르게 나올 수 있기 때문에 조심할것.
rdd = sc.parallelize([1,2,.5, .1, 5,.2], 3)
rdd.reduce(lambda x,y:x/y)

# 0.5   ,  5 , 25
# 0.01  , 25
# 0.004

0.004

In [109]:
rdd.glom().collect()

[[1, 2], [0.5, 0.1], [5, 0.2]]

In [110]:
rdd = sc.parallelize([("a",1), ("b",2), ("a",3)])

__reduceByKey__ : 키 값을 기반으로 value값을 더해서 보여줌.

In [111]:
from operator import add
sorted(rdd.reduceByKey(add).collect())

[('a', 4), ('b', 2)]

__foreach__ : RDD 객체 내의 element들을 한개 씩 python형태로 넘겨주는데 쓰임.
__foreachPartition__ : RDD객체의 partition을 한개 씩 python형태로 넘겨주는데 쓰인다.
__saveAsTextFile__ : RDD객체를 텍스트파일로 저장.

In [142]:
def f(x): print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreach(f)

In [143]:
def f(iterator):
    for x in iterator:
        print(x)
sc.parallelize([1, 2, 3, 4, 5]).foreachPartition(f)

In [148]:
rdd = sc.parallelize([("a",1), ("b",2), ("a",3)], 2)
rdd.saveAsTextFile("abc")
#위에서 partition을 2개로 나눴기 때문에 2개의 파일에 나뉘어져서 rdd 객체가 저장된다.

In [149]:
fromText = sc.textFile("abc") #해당 디렉토리 주소를 인자로 주면 해당파일 불러올 수 있다.
fromText.collect()

["('a', 1)", "('b', 2)", "('a', 3)"]

In [150]:
data = sc.parallelize(
    [("a", 22), {"b": 23}, ["c", 4]]
)

In [151]:
obj = data.collect()

In [152]:
type(obj)

list

In [153]:
obj[1]["b"]

23

RDD는 JVM에서 메모리를 잡고 , python에서는 그냥 통신해서 갖고 오는 것임. 

DataFrame은 데이터 타입 정의된 테이블 형태로 반환(익숙한 형태의 스키마)해주기 때문에,  
row, column에 따라 갖고 올 수 있도록 해준다.

DataFrame 또한 immutable. 수정할 수 없다

DataFrame의 경우 SparkSQL을 쓸 수 있도록 해준다.

In [155]:
from pyspark.sql import SparkSession
# pyspark.sql.SparkSession: DataFrame과 SQL 기능을 쓰기 위한 메인 entry-point(진입점)

In [156]:
spark = SparkSession.builder.getOrCreate()
# 존재하는 SparkSession을 가져오거나, 없다면 새로 생성한다.

In [157]:
spark

In [158]:
sc = SparkContext.getOrCreate()

In [159]:
sc

<SparkContext master=local[2] appName=Conf>

위의 SparkSession을 만들던 SparkContext를 만들던 어짜피 쓰는 드라이버는 똑같다.  
껍데기만 다를 뿐이다.